<a href="https://colab.research.google.com/github/allenyeh929/generative_ai/blob/main/hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.15.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [4]:
import aisuite as ai

In [5]:
provider_writer = "groq"
model_writer ="llama3-70b-8192"

#第二個模型使用 gemma2-9b-it
provider_reply = "groq"
model_reply = "gemma2-9b-it"

In [6]:
def reply(system="請嚴格使用台灣習慣的繁體中文回覆。",
          prompt="Hi",
          provider="groq",
          model="gemma2-9b-it"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

In [8]:
system_writer = """
請嚴格使用台灣習慣的繁體中文回覆。
請逐一回答以下三個角色的想法：

1. 如果我是偵探（訊息接收者），我會怎麼想？
2. 如果我是旁觀者（無利益關係者），我會怎麼看？
3. 如果是兇手，背後的心理動機可能是什麼？

請依序清楚列出這三種視角下的想法，每一點用 1~3 句話說明。
不要進行總結，也不要直接改寫內容，僅負責分角色思考。"""

system_reply = """
請嚴格使用台灣習慣的繁體中文回覆。

根據 system_writer 給的三種視角（偵探、旁觀者、兇手）的思考內容，請撰寫一篇第一人稱視角的全局詮釋短文，統整不同角色的觀點。

- 你可以選擇以「我」為主體來說明整體事件的多重意涵。
- 嘗試指出事件背後可能潛藏的情緒、誤解、動機或衝突。
- 不需逐條照搬角色想法，但需明顯融合這些角度的洞察，呈現更全面的理解。
- 請在文章結尾使用一到兩句話收尾，帶出「事情不單單是表面看到的」這種反思感，讓讀者意識到事件的複雜性。

不要列點，也不需要額外加上「角色分析如下」等標題，只需產出一篇自然、深入、有層次的短文。"""


In [9]:
def rewrite_post(prompt):
    writing_prompt = f"使用者說：{prompt}。請依照三個角色視角分別思考。"
    viewpoints = reply(system_writer, writing_prompt,
                          provider = provider_writer,
                          model = model_writer
                          )

    generation_prompt = f"三種角色視角：\n{viewpoints}\n\n根據這些視角，重寫＋全局詮釋。"
    rewrite_post = reply(system_reply, generation_prompt,
                       provider = provider_reply,
                       model = model_reply
                       )

    return viewpoints, rewrite_post

In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.3 MB/s eta 0:00:00


In [11]:
import gradio as gr

In [12]:
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️‍♂️ 多角色視角思考器：偵探 × 旁觀者 × 兇手")
    gr.Markdown("輸入一段事件或對話，AI 會從偵探、旁觀者、與兇手的角度進行思考，最後生成具洞察力的全局重構短文。")

    user_input = gr.Textbox(label="輸入事件或對話（例如：某人在案發現場說了一句話）", lines=3, placeholder="例如：有人低聲說『反正他活該』...")
    btn = gr.Button("進行角色視角分析 ➔ 全局重寫")

    with gr.Row():
        out1 = gr.Textbox(label="思考階段：三種角色視角", lines=10)
        out2 = gr.Textbox(label="產文階段：全局詮釋短文", lines=10)

    btn.click(rewrite_post, inputs=[user_input], outputs=[out1, out2])

In [13]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://668955db1efa765c6f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://668955db1efa765c6f.gradio.live
